# Geometry documentation

This notebook illustrates the geometry of the system

## 0. Loading packages

In [1]:
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

import pyRaven as rav

loading specpolFlow package


Functions for visualization

In [2]:
def viz_plotly( ROT, c, **kwarg ):

  fig = px.scatter_3d(x=ROT[0,:], y=ROT[1,:], z=ROT[2,:],
                    color=c, **kwarg)
  #fig.show()
  return(fig)

## 1. grid

In [7]:
genparam = {
    'lambda0':5000,    # the central wavelength of the transition
    'vsini':10.0,         # the projected rotational velocity
    'vdop':10.0,          # the thermal broadening
    'av':0.05,             # the damping coefficient of the Voigt profile
    'bnu':1.5,             # the slope of the source function with respect to vertical optical depth
    'logkappa':0.5,          # the line strength parameter
    'ndop':int(10),       # the number of sample point per doppler width for the wavelength array
    'Bpole':1000,
    'incl':90, #not necessary for loop
    'beta':90, #not necessary for loop
    'phase':0 #not necessary for loop
  }

unnoparam = {
    'down':[0, 0, 0],   # the s, j, l of the lower level
    'up':[0, 1, 1],    # the s, j, l of the upper level
    }
    
param={'general' : genparam,
       'unno': unnoparam 
       }


In [8]:
ngrid = rav.diskint2.get_ngrid(param['general']['vsini'], verbose=True)
ROT, A = rav.diskint2.grid_ROT(ngrid) 
los2rot = rav.diskint2.get_los2rot(param['general']['incl'], param['general']['phase'])
rot2los = np.transpose(los2rot)
LOS = rav.diskint2.get_LOS(ROT, rot2los)
mu_LOS, vis, A_LOS, uLOS, conti_flux = rav.diskint2.get_LOS_values(LOS, A, param['general']['bnu'], param['general']['vsini'], param['general']['vdop'])

rot2mag = rav.diskint2.get_rot2mag(param['general']['beta']) 
mag2rot = np.transpose(rot2mag)
MAG = rav.diskint2.get_MAG(ROT, rot2mag)
B_MAG = rav.diskint2.get_B_MAG(MAG)
B_LOS = rav.diskint2.get_B_LOS(B_MAG, mag2rot, rot2los)

B, costheta_temp, sintheta_temp, cos2chi_temp, sin2chi_temp = rav.diskint2.get_unno_angles(B_LOS, param['general']['Bpole'])

Using 1400.0 grid point on the surface


In [10]:
fig = viz_plotly( LOS, sintheta_temp**2*sin2chi_temp, 
        #range_color=[-1,1], 
        title='B Z MAG',
        color_continuous_scale='Picnic' )
camera = dict(
    up=dict(x=0, y=1., z=0),
    eye=dict(x=0., y=0., z=2.5)
)
fig.update_layout(scene_camera=camera)
fig.show()